Reading data & creating an appropriate dataframe

In [227]:
import pandas as pd

movies_df = pd.read_csv('./resources/movies.csv')
ratings_df = pd.read_csv('./resources/ratings.csv')

df = pd.merge(ratings_df, movies_df, on='movieId', how='inner')
# print(df.shape)
# df.head()
movies_df[movies_df.title == 'American Pie (1999)']

,movieId,title,genres
2031,2706,American Pie (1999),Comedy|Romance


Let's narrow the data pool down to movies relevant enough to be rated atleast 100 times

In [228]:
agg_ratings = ratings_df.groupby('movieId').movieId.count().sort_values(ascending=False)
agg_ratings_gt100 = agg_ratings[agg_ratings.values > 100]
df_gt100 = df[df.movieId.isin(agg_ratings_gt100.index)]
print(df_gt100.shape)
df_gt100.head()

(19788, 6)


,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


Let's calculate movie similarities in the form of a matrix using Pearson's correlation

In [229]:
matrix = df_gt100.pivot(index='movieId',columns='userId', values='rating')
# Let's normalize matrix
matrix_norm = matrix.subtract(matrix.mean(axis=1), axis=0)
print(matrix_norm.shape)
matrix_norm.head()

(134, 597)


userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.079070,NaN,NaN,NaN,0.07907,NaN,0.57907,NaN,NaN,NaN,...,0.07907,NaN,0.079070,-0.920930,0.079070,-1.420930,0.07907,-1.420930,-0.920930,1.079070
2,NaN,NaN,NaN,NaN,NaN,0.568182,NaN,0.568182,NaN,NaN,...,NaN,0.568182,NaN,1.568182,0.068182,NaN,NaN,-1.431818,NaN,NaN
6,0.053922,NaN,NaN,NaN,NaN,0.053922,NaN,NaN,NaN,NaN,...,NaN,-0.946078,0.053922,-0.946078,NaN,NaN,NaN,NaN,NaN,1.053922
10,NaN,NaN,NaN,NaN,NaN,-0.496212,NaN,-1.496212,NaN,NaN,...,NaN,-0.496212,NaN,NaN,NaN,NaN,NaN,0.503788,0.503788,NaN
32,NaN,NaN,NaN,-1.983051,NaN,0.016949,NaN,-0.983051,NaN,NaN,...,NaN,-0.983051,-0.983051,0.016949,NaN,0.016949,NaN,-0.483051,NaN,0.516949


In [230]:
similar_movies = matrix.T.corr()
print(similar_movies.shape)
similar_movies.head()


(134, 134)


movieId,1,2,6,10,32,34,39,47,50,110,...,7153,7361,7438,8961,33794,48516,58559,60069,68954,79132
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.330978,0.106465,-0.021409,0.120087,0.407493,0.144193,0.222006,0.250781,0.028768,...,0.101284,0.049218,0.093972,0.643301,0.285349,0.326149,0.255290,0.237234,0.355424,0.076195
2,0.330978,1.000000,0.163510,0.016626,0.084588,0.330327,0.241754,0.088477,0.045364,0.311665,...,0.133258,0.164373,0.056481,0.460369,0.250228,0.274094,0.025349,-0.003548,0.296362,0.300878
6,0.106465,0.163510,1.000000,0.420222,0.184299,-0.005211,-0.169698,0.388253,0.201409,0.327672,...,0.088740,0.103805,-0.174723,0.074621,-0.004642,0.113802,0.143009,0.202412,0.164610,0.565437
10,-0.021409,0.016626,0.420222,1.000000,-0.160238,-0.281090,0.076431,0.243948,0.045645,0.060961,...,0.090872,0.346523,-0.041482,0.094992,0.162075,0.088284,0.152039,0.235220,-0.348508,-0.047223
32,0.120087,0.084588,0.184299,-0.160238,1.000000,0.238554,-0.308569,0.285574,0.255347,0.325520,...,0.312204,0.096976,0.421061,0.256342,0.217525,0.336010,0.350385,0.513187,0.512313,0.472037


Let's pick a user "picked_userId" as the user who has to get recommended movies & for now, just collect all the movies he/she has already watched

In [231]:
picked_userId = 1

watched_movies = pd.Series([index for index in matrix_norm.index if pd.isna(matrix_norm.loc[index,picked_userId]) == False])
watched_movies = matrix_norm[matrix_norm.index.isin(watched_movies.values)].loc[0:,picked_userId]
print(watched_movies.shape)
watched_movies.head()

(56,)


movieId
1      0.079070
6      0.053922
47     1.024631
50     0.762255
110   -0.031646
Name: 1, dtype: float64

Now, let's find the movies user has not watched

In [232]:
notwatched_movies = matrix_norm.loc[~matrix_norm.index.isin(watched_movies.index),picked_userId]
print(notwatched_movies.shape)
notwatched_movies.head()

(78,)


movieId
2    NaN
10   NaN
32   NaN
34   NaN
39   NaN
Name: 1, dtype: float64

Finally, to finish the movie recommendation system, let's follow the following steps:

    1. Loop through unwatched movies & create predicted score for         each movie

    2. Rank the movies based on predicted score
    
    3. Select the top "k" movies to be recommended

In [233]:
def get_score(movie_id, watched_movies):
    total = 0
    count = 0
    #Selecting the movies with highest similarity to movieId from watched movies
    watched_movies_top5 = pd.merge(watched_movies, similar_movies[movie_id], on='movieId', how='inner')
    watched_movies_top5.columns = ['rating','similarity_score']
    watched_movies_top5 = watched_movies_top5.sort_values('similarity_score',ascending=False)[:5]
    # print(watched_movies_top5)

    #Now to get the average weighted score for the top 5 movies
    for id in watched_movies_top5.index:
        score = watched_movies_top5['rating'][id] * watched_movies_top5['similarity_score'][id]
        total += score
        count += 1
        # print(f"{count}: {watched_movies_top5.rating[id]} x {watched_movies_top5.similarity_score[id]} is {score} ")

    # print(watched_movies_top5)
    return total/count
    # print(round(total/count ,6))

In [234]:
# similar_movies[2]
# get_score(2, watched_movies)

In [235]:
k = 10

movie_scores = {}
for movie_id in notwatched_movies.index:
    score = get_score(movie_id, watched_movies)
    movie_scores[movie_id] = score

similar_movies_byScore = pd.DataFrame(movie_scores.items(), columns=['movieId','scores']).sort_values('scores',ascending=False)[0:k]
recommended_movies = pd.merge(similar_movies_byScore, movies_df, how='inner', on='movieId')
recommended_movies

,movieId,scores,title,genres
0,2683,0.480828,Austin Powers: The Spy Who Shagged Me (1999),Action|Adventure|Comedy
1,4973,0.447659,"Amelie (Fabuleux destin d'Amélie Poulain, Le) ...",Comedy|Romance
2,3996,0.428539,"Crouching Tiger, Hidden Dragon (Wo hu cang lon...",Action|Drama|Romance
3,1704,0.409845,Good Will Hunting (1997),Drama|Romance
4,4995,0.374269,"Beautiful Mind, A (2001)",Drama|Romance
5,7153,0.372982,"Lord of the Rings: The Return of the King, The...",Action|Adventure|Drama|Fantasy
6,1221,0.371604,"Godfather: Part II, The (1974)",Crime|Drama
7,1923,0.351053,There's Something About Mary (1998),Comedy|Romance
8,454,0.334599,"Firm, The (1993)",Drama|Thriller
9,586,0.326685,Home Alone (1990),Children|Comedy
